# Gibbs free energy expressions for IN625 from ternary CALPHAD DB

This notebook extracts relevant thermodynamic functions necessary for the phase-field model of solid-state transformations in additive superalloy 625, represented as a ternary (Cr-Nb-Ni) with γ, δ, μ, and Laves phases competing. The thermodynamic database was prepared by U. Kattner after

    @Article{Du2005,
        Title                    = {A thermodynamic modeling of the Cr–Nb–Ni system },
        Author                   = {Yong Du and Shuhong Liu and Y.A. Chang and Ying Yang},
        Journal                  = {Calphad},
        Year                     = {2005},
        Volume                   = {29},
        Number                   = {2},
        Pages                    = {140 - 148},
        Doi                      = {10.1016/j.calphad.2005.06.001}
    }

This database models the phases of interest as follows:
- γ as $\mathrm{(Cr, Nb, Ni)}$
- δ as $\mathrm{(\mathbf{Nb}, Ni)_1(Cr, Nb, \mathbf{Ni})_3}$
- μ as $\mathrm{Nb_6(Cr, Nb, Ni)_7}$
- Laves as $\mathrm{(Cr, \mathbf{Nb})_1(\mathbf{Cr}, Nb, Ni)_2}$

The phase field model requires Gibbs free energies as functions of system compositions $x_\mathrm{Cr}$, $x_\mathrm{Nb}$, $x_\mathrm{Ni}$. The Calphad database represents these energies as functions of sublattice compositions $y$ in each phase. To avoid solving for internal phase equilibrium at each point in the simulation, approximations have been made to allow the following one-to-one mappings between $x$ and $y$:
- γ: no changes necessary
    * $y_\mathrm{Cr}' = x_\mathrm{Cr}$     with $0\leq x_\mathrm{Cr}\leq1$
    * $y_\mathrm{Nb}' = x_\mathrm{Nb}$     with $0\leq x_\mathrm{Nb}\leq1$
    * $y_\mathrm{Ni}' = x_\mathrm{Ni}$     with $0\leq x_\mathrm{Ni}\leq1$
- δ: eliminate Nb from the second (Ni) sublattice, $\mathrm{(\mathbf{Nb}, Ni)_1(Cr, \mathbf{Ni})_3}$
    * $y_\mathrm{Nb}'  = 4x_\mathrm{Nb}$     with $0\leq x_\mathrm{Nb}\leq\frac{1}{4}$
    * $y_\mathrm{Ni}'  = 1-4x_\mathrm{Nb}$   with $0\leq x_\mathrm{Nb}\leq\frac{1}{4}$
    * $y_\mathrm{Cr}'' = \frac{4}{3}x_\mathrm{Cr}$   with $0\leq x_\mathrm{Cr}\leq\frac{3}{4}$
    * $y_\mathrm{Ni}'' = 1-\frac{4}{3}x_\mathrm{Cr}$ with $0\leq x_\mathrm{Cr}\leq\frac{3}{4}$
- μ: no changes necessary
    * $y_\mathrm{Nb}'  = 1$
    * $y_\mathrm{Cr}'' = x_\mathrm{Cr}$      with $0\leq x_\mathrm{Cr}\leq1$
    * $y_\mathrm{Nb}'' = \frac{13}{7}x_\mathrm{Nb}-\frac{6}{7}$ with $\frac{6}{13}\leq x_\mathrm{Nb}\leq1$
    * $y_\mathrm{Ni}'' = x_\mathrm{Ni}$      with $0\leq x_\mathrm{Ni}\leq1$
- Laves: eliminate Nb from the second (Cr) sublattice, $\mathrm{(Cr, \mathbf{Nb})_1(\mathbf{Cr}, Ni)_2}$
    * $y_\mathrm{Cr}' = 1-3x_\mathrm{Nb}$   with $0\leq x_\mathrm{Nb}\leq\frac{1}{3}$
    * $y_\mathrm{Nb}' = 3x_\mathrm{Nb}$   with $0\leq x_\mathrm{Nb}\leq\frac{1}{3}$
    * $y_\mathrm{Cr}'' = 1-\frac{3}{2}x_\mathrm{Ni}$ with $0\leq x_\mathrm{Ni}\leq\frac{2}{3}$
    * $y_\mathrm{Ni}'' = \frac{3}{2}x_\mathrm{Ni}$ with $0\leq x_\mathrm{Ni}\leq\frac{2}{3}$

In [ ]:
%matplotlib inline
# Numerical libraries
import numpy as np
from scipy.optimize import fsolve
from sympy.utilities.lambdify import lambdify
from scipy.spatial import ConvexHull

# Runtime / parallel libraries
import time, warnings
from itertools import chain
from multiprocessing import Pool

# Thermodynamics and computer-algebra libraries
from pycalphad import Database, calculate, Crdel
from sympy.utilities.codegen import codegen
from sympy.parsing.sympy_parser import parse_expr
from sympy import And, diff, Gt, Lt, Piecewise, simplify, symbols, sympify

# Visualization libraries
import matplotlib.pylab as plt
from ipywidgets import FloatProgress
from IPython.display import display

# Constants
fr13by7 = 13.0/7
fr13by3 = 13.0/3
fr13by4 = 13.0/4
fr3by4 = 3.0/4
rt3by2 = np.sqrt(3.0)/2
epsilon = 1e-8 # tolerance for comparing floating-point numbers to zero
temp = 870.0 + 273.15 # 1143 Kelvin
#temp = 1170.0 + 273.15 # Kelvin

RT = 8.3144598*temp # J/mol/K

In [ ]:
#tdb = Database('mc_ni_v2.033.tdb')
#tdb = Database('monbni-simple.tdb')
#tdb = Database('Thermotech_Cr-Nb-Ni_FCC-delta-mu-BCC.tdb')
#tdb = Database('Thermotech_Cr-Nb-Ni_FCC-delta-mu-BCC_simple.tdb')
tdb = Database('Du_Cr-Nb-Ni_simple.tdb')

In [ ]:
phases = ['FCC_A1', 'DELTA', 'MU_PHASE', 'BCC_A2', 'LIQUID']
elements = ['MO', 'NB', 'NI']

## Export Gibbs functions using SymPy

In [ ]:
c_gamma = list(set([i for c in tdb.phases['FCC_A1'].constituents for i in c]))
m_gamma = Crdel(tdb, c_gamma, 'FCC_A1')
g_gamma = parse_expr(str(m_gamma.ast))

In [ ]:
c_delta = list(set([i for c in tdb.phases['DELTA'].constituents for i in c]))
m_delta = Crdel(tdb, c_delta, 'DELTA')
g_delta = parse_expr(str(m_delta.ast))

In [ ]:
c_mu = list(set([i for c in tdb.phases['MU_PHASE'].constituents for i in c]))
m_mu = Crdel(tdb, c_mu, 'MU_PHASE')
g_mu = parse_expr(str(m_mu.ast))

In [ ]:
c_bcc = list(set([i for c in tdb.phases['BCC_A2'].constituents for i in c]))
m_bcc = Crdel(tdb, c_bcc, 'BCC_A2')
g_bcc = parse_expr(str(m_bcc.ast))

## Convert sublattice to phase composition (y to x)

In [ ]:
# Declare sublattice variables used in Pycalphad expressions
FCC_A10MO, FCC_A10NB, FCC_A10NI, FCC_A11VA = symbols('FCC_A10MO FCC_A10NB FCC_A10NI FCC_A11VA')
DELTA0NI, DELTA0NB, DELTA1MO, DELTA1NB, DELTA1NI = symbols('DELTA0NI DELTA0NB DELTA1MO DELTA1NB DELTA1NI')
MU_PHASE0NI, MU_PHASE1MO, MU_PHASE1NB, MU_PHASE2MO, MU_PHASE2NB, MU_PHASE2NI = symbols('MU_PHASE0NI MU_PHASE1MO MU_PHASE1NB MU_PHASE2MO MU_PHASE2NB MU_PHASE2NI')
BCC_A20MO, BCC_A20NB, BCC_A20NI, BCC_A21VA = symbols('BCC_A20MO BCC_A20NB BCC_A20NI BCC_A21VA')
T = symbols('T')

# Declare system variables for target expressions
GAMMA_XMO, GAMMA_XNB, GAMMA_XNI = symbols('GAMMA_XMO GAMMA_XNB GAMMA_XNI')
DELTA_XMO, DELTA_XNB, DELTA_XNI = symbols('DELTA_XMO DELTA_XNB DELTA_XNI')
MU_XMO, MU_XNB, MU_XNI = symbols('MU_XMO MU_XNB MU_XNI')
BCC_XMO, BCC_XNB, BCC_XNI = symbols('BCC_XMO BCC_XNB BCC_XNI')

In [ ]:
# Make substitutions
        
g_gamma = g_gamma.subs({FCC_A10MO: GAMMA_XMO,
                        FCC_A10NB: GAMMA_XNB,
                        FCC_A10NI: GAMMA_XNI,
                        FCC_A11VA: 1,
                        T: temp})

# Approximate delta as a stoichiometric compound, (Nb,Ni)3(Cr,Nb,Ni) ==> Ni3(Cr,Nb,Ni)
g_delta = Piecewise((g_delta.subs({DELTA0NI: 1.0,
                     DELTA0NB: 0,
                     DELTA1MO: 4.0*DELTA_XMO,
                     DELTA1NB: 4.0*DELTA_XNB,
                     DELTA1NI: 4.0*(DELTA_XNI-fr3by4),
                     T: temp}), \
                     Lt(DELTA_XMO+DELTA_XNB, 0.25)),
                    (0, True))

# Substitution made possible by setting y3_Cr=y4_Cr
g_mu = Piecewise((g_mu.subs({MU_PHASE0NI: 1.0,
                  MU_PHASE1NB: 1.0 - fr13by7*MU_XMO,
                  #MU_PHASE1NB: fr13by4*(MU_XNB-MU_XNI) + (39.0/28)*MU_XMO - 9.0/4,
                  MU_PHASE1MO: fr13by7*MU_XMO,
                  MU_PHASE2MO: fr13by7*MU_XMO,
                  MU_PHASE2NB: 3.0 - fr13by7*MU_XMO - fr13by3*MU_XNI,
                  MU_PHASE2NI: fr13by3*MU_XNI - 2.0,
                  T: temp}), \
                  And(Gt(3.0 - fr13by7*MU_XMO - fr13by3*MU_XNI,0.), Gt(fr13by3*MU_XNI,2.))),
                (0, True))

g_bcc = g_bcc.subs({BCC_A20MO: BCC_XMO,
                    BCC_A20NB: BCC_XNB,
                    BCC_A20NI: BCC_XNI,
                    BCC_A21VA: 1.0,
                    T: temp})

In [ ]:
# Generate numerically efficient system-composition expressions
Gg = lambdify((GAMMA_XMO, GAMMA_XNB, GAMMA_XNI), g_gamma, modules='numpy')
Gd = lambdify((DELTA_XMO, DELTA_XNB, DELTA_XNI), g_delta, modules='numpy')
Gu = lambdify((MU_XMO, MU_XNB, MU_XNI), g_mu, modules='numpy')
Gb = lambdify((BCC_XMO, BCC_XNB, BCC_XNI), g_bcc, modules='numpy')

In [ ]:
labels = [r'$\gamma$', r'$\delta$', r'$\mu$', 'BCC']
colors = ['red', 'green','blue', 'yellow']

# plot Gibbs energy curves for fixed x_Cr = 1% to check mu stability
if True:
    xmo = 0.01
    X = []
    G = [[],[],[],[]]

    for xnb in np.linspace(epsilon,1,1001):
        xni = 1-xmo-xnb
        if xni>0:
            X.append(xnb)
            G[0].append(Gg(xmo,xnb,xni))
            G[1].append(Gd(xmo,xnb,xni))
            G[2].append(Gu(xmo,xnb,xni))
            G[3].append(Gb(xmo,xnb,xni))

    plt.figure(figsize=(8, 8))
    plt.xlabel(r'$x_\mathrm{Nb}$', fontsize=24)
    plt.ylabel(r'$\mathcal{G}$', fontsize=24)
    plt.ylim([-100000, -40000])
    for i in range(len(labels)):
        plt.plot(X, G[i], color=colors[i], label=labels[i])
    plt.legend(loc='best')

In [ ]:
# plot Gibbs energy curves for fixed x_Nb = 5% to check delta stability
if True:
    xnb = 0.05
    X = []
    G = [[],[],[],[]]

    for xmo in np.linspace(epsilon,1,1001):
        xni = 1-xmo-xnb
        if xni>0:
            X.append(xmo)
            G[0].append(Gg(xmo,xnb,xni))
            G[1].append(Gd(xmo,xnb,xni))
            G[2].append(Gu(xmo,xnb,xni))
            G[3].append(Gb(xmo,xnb,xni))

    plt.figure(figsize=(8, 8))
    plt.xlabel(r'$x_\mathrm{Cr}$', fontsize=24)
    plt.ylabel(r'$\mathcal{G}$', fontsize=24)
    plt.ylim([-100000, -40000])
    for i in range(len(labels)):
        plt.plot(X, G[i], color=colors[i], label=labels[i])
    plt.legend(loc='best')

# Generate a phase diagram
Using scipy.spatial.ConvexHull, an interface to qhull. This method cannot provide phase fractions, chemical potentials, etc., but will quickly produce the correct diagram from the given Gibbs energies.

In [ ]:
# triangle bounding the Gibbs simplex
XS = [0.0, 1.0, 0.5, 0.0]
YS = [0.0, 0.0,rt3by2, 0.0]
# triangle bounding three-phase coexistence
XT = [0.25, 0.4875+0.025/2,0.5375+0.4625/2, 0.25]
YT = [0.0,  0.025*rt3by2, 0.4625*rt3by2, 0.0]

In [ ]:
def computeKernelExclusive(n):
    a = n / density # index along x-axis
    b = n % density # index along y-axis

    xnb = epsilon + 1.0*a / (density-1)
    xmo = epsilon + 1.0*b / (density-1)
    xni = 1.0 - xmo - xnb

    result = [0]*7
    
    if xni>0:
        result[0] = xmo
        result[1] = xnb
        result[2] = xni
        result[3] = Gg(xmo,xnb,xni)
        result[4] = Gd(xmo,xnb,xni)
        result[5] = Gu(xmo,xnb,xni)
        result[6] = Gb(xmo,xnb,xni)
    
    return result

In [ ]:
density = 201
allCr = []
allNb = []
allG = []
allID = []
points = []
phases = []

if __name__ == '__main__':
    starttime = time.time() # not exact, but multiprocessing makes time.clock() read from different cores

    bar = FloatProgress(min=0,max=density**2)
    display(bar)

    pool = Pool(12)

    i = 0
    for result in pool.imap(computeKernelExclusive, range(density**2)):
        xmo, xnb, xni, fg, fd, fu, fb = result
        f = (fg, fd, fu, fb)

        # Accumulate (x, y, G) points for each phase
        if (fd**2 + fu**2 + fb**2) > epsilon:
            for n in range(len(f)):
                allCr.append(rt3by2*xmo)
                allNb.append(xnb+xmo/2)
                allG.append(f[n])
                allID.append(n)
        i += 1
        bar.value = i

    pool.close()
    pool.join()
    
    points = np.array([allCr, allNb, allG]).T
    
    hull = ConvexHull(points)
    
    runtime = time.time() - starttime
    print "%ih:%im:%is elapsed" % (int(runtime/3600), int(runtime/60)%60, int(runtime)%60)

In [ ]:
# Prepare arrays for plotting
labels = [r'$\gamma$', r'$\delta$', r'$\mu$', 'BCC']
colors = ['red', 'green','blue', 'yellow']
X = [[],[],[],[]]
Y = [[],[],[],[]]
tielines = []
trilines = []

for simplex in hull.simplices:
    for i in simplex:
        X[allID[i]].append(allNb[i])
        Y[allID[i]].append(allCr[i])
        for j in simplex:
            if allID[i]==allID[j]:
                pass
            tielines.append([[allNb[i], allNb[j]],
                             [allCr[i], allCr[j]]])

In [ ]:
# Plot phase diagram
pltsize = 20
plt.figure(figsize=(pltsize, rt3by2*pltsize))
plt.title("Cr-Nb-Ni at %.0fK"%temp, fontsize=18)
plt.xlim([0,1])
plt.ylim([0,rt3by2])
plt.xlabel(r'$x_\mathrm{Nb}$', fontsize=24)
plt.ylabel(r'$x_\mathrm{Cr}$', fontsize=24)
plt.plot(XS, YS, '-k')
for tie in tielines:
    plt.plot(tie[0], tie[1], '-k', alpha=0.02)
for i in range(len(labels)):
    plt.scatter(X[i], Y[i], color=colors[i], s=2.5, label=labels[i])
plt.legend(loc='best')
plt.show()

## Interfacial Energy
This is a naïve attempt to determine an appropriate interfacial energy for $\mu$-phase by finding the balancing point of metastable equilibrium between $\gamma$ and $\mu$.

In [ ]:
sigma_del = 1.01 # J/m**2
sigma_mu = sigma_del / 20 # J/m**2
rmax = 1.0e-8  # 1 mm
rmin = 1.0e-14 # 0.1 μm
Vm = 1.0e-5 # m**3/mol

def computeKernelInterfacial(n):
    a = n / density # index along x-axis
    b = n % density # index along y-axis

    m = (1.0 - epsilon)/(density - 1)
    xnb = m * a + epsilon
    
    m = (rmax - rmin) / (density - 1)
    r = rmin + m * b

    xmo = 0.01 # constant, midpoint of mu phase field
    xni = 1 - xmo - xnb

    result = [0]*7
    
    if xni>0:
        result[0] = b
        result[1] = xnb
        result[2] = xni
        result[3] = Gg(xmo,xnb,xni)
        result[4] = Gd(xmo,xnb,xni) + 2.0 * Vm * sigma_del / r
        result[5] = Gu(xmo,xnb,xni) + 2.0 * Vm * sigma_mu / r
        result[6] = Gb(xmo,xnb,xni)

    return result

In [ ]:
labels = [r'$\gamma$', r'$\delta$', r'$\mu$', 'BCC']
colors = ['red', 'green','blue', 'yellow']

# plot Gibbs energy curves for fixed x_Cr = 1% to check mu stability
if True:
    xmo = 0.01
    X = []
    G = [[],[],[],[]]
    radius = 1.0e-4

    for xnb in np.linspace(epsilon,1,1001):
        xni = 1-xmo-xnb
        if xni>0:
            X.append(xnb)
            G[0].append(Gg(xmo,xnb,xni))
            G[1].append(Gd(xmo,xnb,xni) + 2.0 * Vm * sigma_del / radius)
            G[2].append(Gu(xmo,xnb,xni) + 2.0 * Vm * sigma_mu / radius)
            G[3].append(Gb(xmo,xnb,xni))

    plt.figure(figsize=(8, 8))
    plt.xlabel(r'$x_\mathrm{Nb}$', fontsize=24)
    plt.ylabel(r'$\mathcal{G}$', fontsize=24)
    plt.ylim([-80000, -50000])
    for i in range(len(labels)):
        plt.plot(X, G[i], color=colors[i], label=labels[i])
    plt.legend(loc='best')

In [ ]:
density = 1001
allR = []
allNb = []
allG = []
allID = []
points = []
phases = []


if __name__ == '__main__':
    starttime = time.time() # not exact, but multiprocessing makes time.clock() read from different cores

    bar = FloatProgress(min=0,max=density**2) #(density-1)/2) 
    display(bar)

    pool = Pool(12)

    i = 0
    for result in pool.imap(computeKernelInterfacial, range(density**2)):
        r, xnb, xni, fg, fd, fu, fb = result

        # Accumulate (x, G) points for each phase
        if xni>0:
            f = (fg, fd, fu, fb)
            for n in range(len(f)):
                allR.append(r)
                allNb.append(xnb)
                allG.append(f[n])
                allID.append(n)
        i += 1
        bar.value = i

    pool.close() # not adding any more processes
    pool.join() # wait until all threads are done before going on
        
    
    runtime = time.time() - starttime
    print "%ih:%im:%is elapsed" % (int(runtime/3600), int(runtime/60)%60, int(runtime)%60)

In [ ]:
# Prepare arrays for plotting
X = [[],[],[],[]]
Y = [[],[],[],[]]
tielines = []

for r in range(density):
    myI = []
    myR = []
    myN = []
    myG = []
    for n in range(len(allR)):
        if allR[n]==r:
            myI.append(allID[n]) # phase ID
            myR.append(rmin + allR[n] * (rmax - rmin) / (density - 1))  # radius
            myN.append(allNb[n]) # Nb conc
            myG.append(allG[n])  # Gibbs energy
    points = np.array([myN, myG]).T
    hull = ConvexHull(points)

    for simplex in hull.simplices:
        for i in simplex:
            Y[myI[i]].append(myR[i])
            X[myI[i]].append(myN[i])
            for j in simplex:
                if myI[i]==myI[j]:
                    pass
                tielines.append([[myN[i], myN[j]],
                                 [myR[i], myR[j]]])

In [ ]:
# Plot phase diagram
pltsize = 20
plt.figure(figsize=(pltsize, rt3by2*pltsize))
plt.title("1%%Cr-Nb-Ni at %.0fK"%temp, fontsize=18)
plt.xlim([0,1])
#plt.ylim([0,1.01*rmax])
plt.xlabel(r'$x_\mathrm{Nb}$', fontsize=24)
plt.ylabel(r'$r$', fontsize=24)
plt.yscale('log')
for tie in tielines:
    plt.plot(tie[0], tie[1], '-k', alpha=0.02)
for i in range(len(labels)):
    plt.scatter(X[i], Y[i], color=colors[i], s=2.5, label=labels[i])
plt.legend(loc='best')
plt.show()